# Mini-projet n°3 - ViT vs CNN

L'objectif de ce mini-projet est de comparer les performances de deux architectures différentes pour de la classification d'image : les Vision Transformers (ViT) et les Convolutional Neural Networks (CNN).